In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip "/content/drive/MyDrive/updated dataset (2).zip"

Streaming output truncated to the last 5000 lines.
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y1751_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y1801_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y1851_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y1901_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y1951_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y2001_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y2051_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y2101_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1751_y2151_class1.png  
 extracting: updated dataset/IDC_regular_ps50_idx5/9023/1/9023_idx5_x1801_y1251_class1.png  
 extracting: update

In [6]:
import os, shutil
from pip._vendor.rich import progress

parent = "/content/updated dataset/IDC_regular_ps50_idx5"

patients = os.listdir(parent)

positives = []
negatives = []

for patient in patients:
    for pos in os.listdir(f"{parent}/{patient}/1/"):
        positives.append(f"{parent}/{patient}/1/{pos}")
    for neg in os.listdir(f"{parent}/{patient}/0/"):
        negatives.append(f"{parent}/{patient}/0/{neg}")

positive_distribution_pivot = int(len(positives)*0.75)
negative_distribution_pivot = int(len(negatives)*0.75)

os.makedirs("./breastcancer/train/malignant/",exist_ok=True)
os.makedirs("./breastcancer/train/benign/",exist_ok=True)
os.makedirs("./breastcancer/test/malignant/",exist_ok=True)
os.makedirs("./breastcancer/test/benign/",exist_ok=True)

for file in progress.track(positives[:positive_distribution_pivot], description = "Copying To : Dataset/train/malignant/"):
    shutil.copy(file,f"./breastcancer/train/malignant/")
for file in progress.track(positives[positive_distribution_pivot:], description = "Copying To : Dataset/test/malignant/ "):
    shutil.copy(file,f"./breastcancer/test/malignant/")

for file in progress.track(negatives[:negative_distribution_pivot], description = "Copying To : Dataset/train/benign/   "):
    shutil.copy(file,f"./breastcancer/train/benign/")
for file in progress.track(negatives[negative_distribution_pivot:], description = "Copying To : Dataset/test/benign/    "):
    shutil.copy(file,f"./breastcancer/test/benign/")

Output()

Output()

Output()

Output()

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sys
import os
from keras.applications.vgg16 import VGG16
import keras
from numpy import load
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras import backend
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

from keras.layers import Dropout


In [6]:
train_set = "/content/breastcancer/train"
test_set = "/content/breastcancer/test"

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)



In [8]:
batch_size = 128
train_transform = train_datagen.flow_from_directory(train_set,
                                                 target_size=(224, 224),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')



Found 6542 images belonging to 2 classes.


In [9]:
test_transform =test_datagen.flow_from_directory(test_set,
                                            target_size=(224, 224),
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 2182 images belonging to 2 classes.


In [10]:
train_class = train_transform.class_indices
print(train_class)


{'benign': 0, 'malignant': 1}


In [11]:
a= list(train_class.keys())
print(a)

['benign', 'malignant']


In [12]:
train_num = train_transform.samples
test_num = test_transform.samples

In [13]:
base_model=VGG16(include_top=False,input_shape=(224,224,3))
base_model.trainable=False

In [14]:
model=keras.models.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2)                 50178     
                                                                 
Total params: 14,764,866
Trainable params: 50,178
Non-trainable params: 14,714,688
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
train_num//batch_size

51

In [17]:
test_num//batch_size

17

In [21]:
history=model.fit(train_transform,steps_per_epoch = len(train_transform)//10,epochs=5, validation_data=test_transform,validation_steps = len(train_transform)//10)

Epoch 1/5
5/5 [==============================] - 837s 186s/step - loss: 0.5152 - accuracy: 0.8438 - val_loss: 0.4208 - val_accuracy: 0.8359
Epoch 2/5
5/5 [==============================] - 854s 193s/step - loss: 0.3117 - accuracy: 0.8578 - val_loss: 0.4492 - val_accuracy: 0.8016
Epoch 3/5
5/5 [==============================] - 863s 195s/step - loss: 0.2577 - accuracy: 0.9047 - val_loss: 0.3518 - val_accuracy: 0.8531
Epoch 4/5
5/5 [==============================] - 864s 194s/step - loss: 0.2402 - accuracy: 0.9109 - val_loss: 0.4178 - val_accuracy: 0.8453
Epoch 5/5
5/5 [==============================] - 826s 185s/step - loss: 0.2172 - accuracy: 0.9281 - val_loss: 0.3402 - val_accuracy: 0.8828


In [22]:
model.save("breastcancer.h5")

In [23]:
from keras.preprocessing import image
import numpy as np

In [24]:

import seaborn as sns
sns.set()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)



In [25]:
from PIL import Image
import numpy as np


In [36]:
#Testing 1
image_path = "/content/breastcancer/test/malignant/8913_idx5_x401_y1201_class1.png"
new_img = Image.open(image_path).resize((224,224))
img = np.array(new_img) / 255.0
img = np.expand_dims(img, axis=0)
print("Following is our prediction:")
prediction = model.predict(img)
a = ['benign','malignant']
d = prediction.flatten()
j = d.max()
for index, item in enumerate(d):
    if item == j:
        class_name = a[index]
print(class_name)

Following is our prediction:
1/1 [==============================] - 1s 542ms/step
malignant


In [35]:
#Testing 2
image_path = "/content/breastcancer/test/benign/8913_idx5_x1001_y751_class0.png"
new_img = Image.open(image_path).resize((224,224))
img = np.array(new_img) / 255.0
img = np.expand_dims(img, axis=0)

print("Following is our prediction:")
prediction = model.predict(img)
a = ['benign','malignant']

d = prediction.flatten()
j = d.max()
for index, item in enumerate(d):
    if item == j:
        class_name = a[index]
print(class_name)

Following is our prediction:
1/1 [==============================] - 1s 540ms/step
benign
